# yo-yo-maskr

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

'To answer the question "What is LangChain?", let\'s break it down step by step.\n\nLangChain is an open-source, Python-based framework that enables developers to build and integrate blockchain data pipelines into their existing applications. Here\'s a step-by-step explanation:\n\n1. **Blockchain integration**: LangChain allows developers to easily connect their applications with various blockchain platforms, such as Ethereum, Polkadot, or Solana.\n2. **Data pipeline management**: The framework provides tools for building and managing data pipelines that process and transform blockchain data in real-time.\n3. **Smart contract interactions**: LangChain enables seamless interactions with smart contracts on different blockchain networks, allowing developers to automate tasks, execute transactions, and retrieve data from the blockchain.\n4. **Decentralized data storage**: The framework supports decentralized data storage solutions like InterPlanetary File System (IPFS) or Swarm, which enab

In [2]:
with open('sample_text.txt', 'r') as file:
    sample_text = file.read()
sample_text

'Hier ist ein kurzer Text mit vielen Namen von Personen und Orten:\n\n---\n\nEs war ein kühler Herbstmorgen, als Marie Müller mit ihrem Bruder Thomas in Hamburg auf den Zug nach Berlin wartete. Dort wollten sie ihre Freunde Anna Schmidt und Lukas Hoffmann treffen, die gerade aus München zurückgekehrt waren. Während der Reise erzählte Marie von ihrem letzten Urlaub in Italien, wo sie in Rom einen alten Schulfreund namens Peter Lehmann getroffen hatte. Peter wohnte mittlerweile in der Nähe von Mailand und arbeitete für eine Firma in Venedig.\n\nIn Berlin angekommen, holte sie ihr Onkel Karl Weber vom Bahnhof ab. Gemeinsam fuhren sie zu seiner Wohnung in der Friedrichstraße, wo sie später an einem Abendessen bei Familie Becker in Charlottenburg teilnehmen wollten. Dort kamen auch Karla Fischer, eine ehemalige Klassenkameradin, und Paul König, ein alter Freund aus der Schulzeit, hinzu.\n\nAm nächsten Tag machten sich Marie und Thomas auf den Weg nach Potsdam, um dort den berühmten Park San